# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive/CS194-26/Proj2

In [ ]:
%pylab inline

import os
from tqdm.notebook import tqdm
import cv2
import skimage as sk
from skimage import transform
import skimage.io as skio
import matplotlib.pyplot as plt

!pip install line_profiler
%load_ext line_profiler

# Utils

In [ ]:
def get_image(path):
  im = imread(path)
  im = im.astype(float)
  return im

def show(im, figsize = 10, cmap=None):
  # Uses matplotlib in the backend to show images
  figure(figsize=(figsize,figsize))
  imshow(im, cmap=cmap)


# Part 1

Imports

In [ ]:
from scipy.signal import convolve2d

## 1.1 Finite Difference Operator

In [ ]:
cameraman = get_image('cameraman.png')
dX = np.array([[-1, 1]])
dY = np.array([[1], [-1]])


show(cameraman, cmap = 'gray', figsize = 10)

In [ ]:
edge_x = convolve2d(cameraman[:,:,0], dX, mode = 'same', boundary = 'fill')
show(edge_x, cmap = 'gray')
plt.title('D_x')

edge_y = convolve2d(cameraman[:,:,0], dY, mode = 'same', boundary = 'fill')
show(edge_y, cmap = 'gray')
plt.title('D_y')


In [ ]:
edges = np.sqrt(edge_x**2 + edge_y**2)
show(edges, cmap = 'gray')
plt.title('Gradient Magnitude')

For this threshold value, the best is somewhere between .15 and .18

In [ ]:
fig, axs = fig, ax = plt.subplots(5, 2, figsize=(15, 40))
increment = .04
for i in range(10):
  edges = np.sqrt(edge_x**2 + edge_y**2)
  threshold = increment * i
  edges = edges > threshold
  axs[i//2, i % 2].imshow(edges, cmap = 'gray')
  axs[i//2, i % 2].set_title('Threshold: ' + str(increment * i))

fig.show()



## 1.2 DoG Filter

**NOTE**: We are using an odd sized gaussian kernel since convolving D_x * G would otherwise be lopsided due to edge behavior

There aren't any real noticable differences between the original image and this one... just slightly blurrier

To do all this, we just blur the original image, and then do the same derivative filters that we did before

In [ ]:
k_size = 12
G = cv2.getGaussianKernel(k_size, 1) @ cv2.getGaussianKernel(k_size, 1).T
cameraman_blur = convolve2d(cameraman[:,:,0], G, mode = 'same', boundary = 'fill')
show(cameraman_blur, cmap = 'gray')
plt.title('Cameraman Blurred')
edge_x = convolve2d(cameraman_blur, dX, mode = 'same', boundary = 'fill')
edge_y = convolve2d(cameraman_blur, dY, mode = 'same', boundary = 'fill')


In [ ]:
edges = np.sqrt(edge_x**2 + edge_y**2)

show(edges, cmap = 'gray')
plt.title('Gradient Magnitude (Blurred)')

For this threshold value, the best is somewhere between .15 and .18

In [ ]:
fig, axs = fig, ax = plt.subplots(5, 2, figsize=(15, 40))
increment = .01
for i in range(10):
  edges = np.sqrt(edge_x**2 + edge_y**2)
  threshold = increment * i
  edges = edges > threshold
  axs[i//2, i % 2].imshow(edges, cmap = 'gray')
  axs[i//2, i % 2].set_title('Threshold: ' + str(increment * i))

fig.show()



Now Using (G * derivatives) * image

To do this, we will take our even sized gaussian kernel (larger gets better edge behavior) and we will convolve dx (np.array([[1, -1]]) over G and dy (np.array([[1],[-1]])) over G, taking the magnitude of both gradients to get out edges

In [ ]:
G = cv2.getGaussianKernel(k_size, 1) @ cv2.getGaussianKernel(k_size, 1).T
G_dx = convolve2d(G, np.array([[1, -1]]), mode = 'same', boundary = 'fill')
G_dy = convolve2d(G, np.array([[-1],[1]]), mode = 'same', boundary = 'fill')

edge_x = convolve2d(cameraman[:,:,0], G_dx, mode = 'same', boundary = 'fill')
edge_y = convolve2d(cameraman[:,:,0], G_dy, mode = 'same', boundary = 'fill')

Gradient Magnitude Blurred (G * derivatives) * image

Qualitativley, this looks the same

In [ ]:
edges = np.sqrt(edge_x**2 + edge_y**2)
show(edges, cmap = 'gray')
plt.title('Gradient Magnitude (Blurred)')

Thresholding with (G * derivatives) * image

Qualitativley, still looks the same

In [ ]:
fig, axs = fig, ax = plt.subplots(5, 2, figsize=(15, 40))
increment = .01
for i in range(10):
  edges = np.sqrt(edge_x**2 + edge_y**2)
  threshold = increment * i
  edges = edges > threshold
  axs[i//2, i % 2].imshow(edges, cmap = 'gray')
  axs[i//2, i % 2].set_title('Threshold: ' + str(increment * i))

fig.show()

# Part 2

Imports

In [ ]:
from scipy.fft import fft2, ifft2
import align_image_code
from align_image_code import align_images

import matplotlib
%matplotlib
# make align_images call here
%matplotlib inline
#matplotlib.use('TkAgg')


def fft_magnitude(img):
  return np.absolute(fft2(img))






import math
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as sktr



def get_points(im1, im2):
    print('Please select 2 points in each image for alignment.')
    plt.imshow(im1)
    p1, p2 = plt.ginput(2)
    plt.close()
    plt.imshow(im2)
    p3, p4 = plt.ginput(2)
    plt.close()
    return (p1, p2, p3, p4)

def recenter(im, r, c):
    R, C, _ = im.shape
    rpad = (int) (np.abs(2*r+1 - R))
    cpad = (int) (np.abs(2*c+1 - C))
    return np.pad(
        im, [(0 if r > (R-1)/2 else rpad, 0 if r < (R-1)/2 else rpad),
             (0 if c > (C-1)/2 else cpad, 0 if c < (C-1)/2 else cpad),
             (0, 0)], 'constant')

def find_centers(p1, p2):
    cx = np.round(np.mean([p1[0], p2[0]]))
    cy = np.round(np.mean([p1[1], p2[1]]))
    return cx, cy

def align_image_centers(im1, im2, pts):
    p1, p2, p3, p4 = pts
    h1, w1, b1 = im1.shape
    h2, w2, b2 = im2.shape
    
    cx1, cy1 = find_centers(p1, p2)
    cx2, cy2 = find_centers(p3, p4)

    im1 = recenter(im1, cy1, cx1)
    im2 = recenter(im2, cy2, cx2)
    return im1, im2

def rescale_images(im1, im2, pts):
    p1, p2, p3, p4 = pts
    len1 = np.sqrt((p2[1] - p1[1])**2 + (p2[0] - p1[0])**2)
    len2 = np.sqrt((p4[1] - p3[1])**2 + (p4[0] - p3[0])**2)
    dscale = len2/len1
    if dscale < 1:
        im1 = sktr.rescale(im1, dscale, multichannel=True)
    else:
        im2 = sktr.rescale(im2, 1./dscale, multichannel=True)
    return im1, im2

def rotate_im1(im1, im2, pts):
    p1, p2, p3, p4 = pts
    theta1 = math.atan2(-(p2[1] - p1[1]), (p2[0] - p1[0]))
    theta2 = math.atan2(-(p4[1] - p3[1]), (p4[0] - p3[0]))
    dtheta = theta2 - theta1
    im1 = sktr.rotate(im1, dtheta*180/np.pi)
    return im1, dtheta

def match_img_size(im1, im2):
    # Make images the same size
    h1, w1, c1 = im1.shape
    h2, w2, c2 = im2.shape
    if h1 < h2:
        im2 = im2[int(np.floor((h2-h1)/2.)) : -int(np.ceil((h2-h1)/2.)), :, :]
    elif h1 > h2:
        im1 = im1[int(np.floor((h1-h2)/2.)) : -int(np.ceil((h1-h2)/2.)), :, :]
    if w1 < w2:
        im2 = im2[:, int(np.floor((w2-w1)/2.)) : -int(np.ceil((w2-w1)/2.)), :]
    elif w1 > w2:
        im1 = im1[:, int(np.floor((w1-w2)/2.)) : -int(np.ceil((w1-w2)/2.)), :]
    assert im1.shape == im2.shape
    return im1, im2

def align_images(im1, im2):
    pts = get_points(im1, im2)
    im1, im2 = align_image_centers(im1, im2, pts)
    im1, im2 = rescale_images(im1, im2, pts)
    im1, angle = rotate_im1(im1, im2, pts)
    im1, im2 = match_img_size(im1, im2)
    return im1, im2

## 2.1 Image "Sharpening"

Naive sharpen is the way of sharpening an image by doing a bunch of operations, adding the high pass scaled by alpha to the original image

Fast sharpen is the combined version where we do everything in a single convolution

In [ ]:
def naive_sharpen(image, k_size, alpha=.1, g_std = 1):
  G = cv2.getGaussianKernel(k_size, g_std) @ cv2.getGaussianKernel(k_size, g_std).T
  low_pass = convolve2d(image, G, mode='same')
  high_pass = image - low_pass

  return np.clip(image + alpha * high_pass, 0, 1)

def fast_sharpen(image, k_size, alpha=.1, g_std=1):
  G = cv2.getGaussianKernel(k_size, g_std) @ cv2.getGaussianKernel(k_size, g_std).T
  #make an identity conv with just a one in the center... same size as gaussian kernel. Special case for even sized
  # gaussian kernel, since the placement of the one will effect whether or not it is truly the identity conv
  # for a "same" convolution
  if not k_size % 2:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2, k_size//2] = 1
  else:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2 + 1, k_size//2 + 1] = 1

  combined_kernel = (1 + alpha) * identity_conv - alpha * G
  conv = convolve2d(image, combined_kernel, mode = 'same')

  return np.clip(conv, 0, 1)


The control. Just the regular image

In [ ]:
show(cameraman[:,:,0], cmap = 'gray')

Naive sharpening (not done all in one kernel)

In [ ]:
show(naive_sharpen(cameraman[:,:,0], 11, .5), cmap = 'gray')

Clearly the fast sharpening works, since the output is the same as the naive way

In [ ]:
show(fast_sharpen(cameraman[:,:,0], 11, .5), cmap = 'gray')

Working with some custom images

The results are pretty good. Colors arent unbalanced or anything, and background/blurry features come more into view. I suspect though that if I had a larger screen to view these images on, I would find the result less appealing since the sharpening would definitley make smaller features come more into view, distracting me from the primary object in the images

In [ ]:
im = get_image('taj.jpeg')/255
sharpened = np.stack([fast_sharpen(im[:,:,i], 11, .7) for i in range(3)], axis=-1)

fig, axs = plt.subplots(1, 2, figsize=(24, 24))

axs[0].imshow(im, cmap = 'gray')
axs[0].set_title("Unsharpened")
axs[1].imshow(sharpened, cmap = 'gray')
axs[1].set_title("Sharpened")

In [ ]:
im = get_image('beautiful_world.jpg')/255
sharpened = np.stack([fast_sharpen(im[:,:,i], 11, .7) for i in range(3)], axis=-1)

fig, axs = plt.subplots(1, 2, figsize=(24, 24))

axs[0].imshow(im, cmap = 'gray')
axs[0].set_title("Unsharpened")
axs[1].imshow(sharpened, cmap = 'gray')
axs[1].set_title("Sharpened")

In [ ]:
im = get_image('red.jpg')/255
sharpened = np.stack([fast_sharpen(im[:,:,i], 11, .9) for i in range(3)], axis=-1)

fig, axs = plt.subplots(1, 2, figsize=(24, 24))

axs[0].imshow(im, cmap = 'gray')
axs[0].set_title("Unsharpened")
axs[1].imshow(sharpened, cmap = 'gray')
axs[1].set_title("Sharpened")

In [ ]:
im = get_image('miracles.jpg')/255
sharpened = np.stack([fast_sharpen(im[:,:,i], 11, .7) for i in range(3)], axis=-1)

fig, axs = plt.subplots(1, 2, figsize=(24, 24))

axs[0].imshow(im, cmap = 'gray')
axs[0].set_title("Unsharpened")
axs[1].imshow(sharpened, cmap = 'gray')
axs[1].set_title("Sharpened")

In [ ]:
im = get_image('temple.jpg')/255
sharpened = np.stack([fast_sharpen(im[:,:,i], 11, .7) for i in range(3)], axis=-1)

fig, axs = plt.subplots(1, 2, figsize=(24, 24))

axs[0].imshow(im, cmap = 'gray')
axs[0].set_title("Unsharpened")
axs[1].imshow(sharpened, cmap = 'gray')
axs[1].set_title("Sharpened")

In [ ]:
show(sharpened, figsize=50)

## 2.2 Hybrid Images

Here, the idea is that we add the weighted high pass and the weighted low pass, getting our final image

In [ ]:
def hybrid_image(image_high, image_low, k_size=11, alpha=.5, g_std=1):
  G = cv2.getGaussianKernel(k_size, g_std) @ cv2.getGaussianKernel(k_size, g_std).T
  if not k_size % 2:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2, k_size//2] = 1
  else:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2 + 1, k_size//2 + 1] = 1

  high_pass_kernel = identity_conv - (G)
  low_pass_kernel = G

  high_pass = convolve2d(image_high, high_pass_kernel, 'same')
  low_pass = convolve2d(image_low, low_pass_kernel, 'same')

  return np.clip(high_pass * alpha + low_pass * (1-alpha), 0, 1)

def hybrid_uncombined(image_high, image_low, k_size=11, alpha=.5, g_std=1):
  G = cv2.getGaussianKernel(k_size, g_std) @ cv2.getGaussianKernel(k_size, g_std).T
  if not k_size % 2:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2, k_size//2] = 1
  else:
    identity_conv = np.zeros((k_size, k_size))
    identity_conv[k_size//2 + 1, k_size//2 + 1] = 1

  high_pass_kernel = identity_conv - (G)
  low_pass_kernel = G

  high_pass = convolve2d(image_high, high_pass_kernel, 'same')
  low_pass = convolve2d(image_low, low_pass_kernel, 'same')

  return high_pass, low_pass


In [ ]:
im_high = get_image('nutmeg.jpg')/255
im_low = get_image('DerekPicture.jpg')/255

pts = ((752, 373), (607, 287), (445, 333), (305, 348))

im_high, im_low = align_image_centers(im_high, im_low, pts)
im_high, im_low = rescale_images(im_high, im_low, pts)
im_high, angle = rotate_im1(im_high, im_low, pts)
im_high, im_low = match_img_size(im_high, im_low)
show(hybrid_image(np.mean(im_high, -1), np.mean(im_low, -1), alpha=.65, k_size=25, g_std=10), cmap = 'gray')

See the filtering in action!

In [ ]:
im_high = cv2.resize(get_image('pickup.jpeg')/255, (1100, 600))
im_low = cv2.resize(get_image('cybertruck.jpeg')/255, (1100, 600))

im_high_original = im_high
im_low_original = im_low

pts = ((222, 388), (824, 382), (180, 427), (881, 419))

im_high, im_low = align_image_centers(im_high, im_low, pts)
im_high, im_low = rescale_images(im_high, im_low, pts)
im_high, angle = rotate_im1(im_high, im_low, pts)
im_high, im_low = match_img_size(im_high, im_low)

im_high_original = np.mean(im_high, -1)
im_low_original = np.mean(im_low, -1)
show(hybrid_image(np.mean(im_high, -1), np.mean(im_low, -1), alpha=.5, k_size=50, g_std=12), cmap = 'gray')




You can see that our wide gaussian only really filters the middle stuff

In [ ]:
show(cv2.getGaussianKernel(100, 25) @ cv2.getGaussianKernel(100, 25).T)
show((np.abs(np.fft.fftshift(np.fft.fft2(cv2.getGaussianKernel(100, 25) @ cv2.getGaussianKernel(100, 25).T)))))

You can see the large effect of our low pass filter (the bars remaining in the frequency domain are clearly from the fact that our image allignment leaves large black bars on our image that have such high amplitudes, that even a gaussian has difficulty removing them entirely)

In [ ]:
high_pass, low_pass = hybrid_uncombined(np.mean(im_high, -1), np.mean(im_low, -1), alpha=.5, k_size=50, g_std=12)
fig, axs = plt.subplots(2, 4, figsize=(24, 12))
axs[0, 0].imshow(im_high_original, cmap = 'gray')
scale1 = np.max(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_high_original)))))
minscale1 = np.min(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_high_original)))))
axs[0, 1].imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_high_original)))), cmap = 'gray', vmax=scale1, vmin = minscale1)
axs[0, 2].imshow(high_pass, cmap = 'gray')
axs[0, 3].imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(high_pass)))), cmap = 'gray', vmax=scale1, vmin = minscale1)
axs[1, 0].imshow(im_low_original, cmap = 'gray')
scale2 = np.max(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_low_original)))))
minscale2 = np.min(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_low_original)))))
axs[1, 1].imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(im_low_original)))), cmap = 'gray', vmax=scale2, vmin = minscale2)
axs[1, 2].imshow(low_pass, cmap = 'gray')
axs[1, 3].imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(low_pass)))), cmap = 'gray', vmax=scale2, vmin=minscale2)

fig.show()



See some other images become hybrids

In [ ]:
im_high = cv2.resize(get_image('pickup.jpeg')/255, (1100, 600))
im_low = cv2.resize(get_image('cybertruck.jpeg')/255, (1100, 600))

pts = ((222, 388), (824, 382), (180, 427), (881, 419))

im_high, im_low = align_image_centers(im_high, im_low, pts)
im_high, im_low = rescale_images(im_high, im_low, pts)
im_high, angle = rotate_im1(im_high, im_low, pts)
im_high, im_low = match_img_size(im_high, im_low)
show(hybrid_image(np.mean(im_high, -1), np.mean(im_low, -1), alpha=.5, k_size=25, g_std=12), cmap = 'gray')

In [ ]:
im_low = cv2.resize(get_image('cookie1.jpeg')/255, (900, 600))
im_high = cv2.resize(get_image('cookie2.jpeg')/255, (900, 600))

pts = ((14, 271), (479, 266), (274, 304), (645, 313))

im_high, im_low = align_image_centers(im_high, im_low, pts)
im_high, im_low = rescale_images(im_high, im_low, pts)
im_high, angle = rotate_im1(im_high, im_low, pts)
im_high, im_low = match_img_size(im_high, im_low)
show(hybrid_image(np.mean(im_high, -1), np.mean(im_low, -1), alpha=.7, k_size=25, g_std=4), cmap = 'gray')


Bells and Whistles

Color seems to be finnickey. You clearly need some of both, but it can be really challenging to get the color balance while maintaining the image quality. I'm using mostly color from the high frequency image, with a little bit of the color from the low frequency image, but that can certainly be tweaked.

In [ ]:
im = np.stack([hybrid_image(im_high[:,:,i], im_low[:,:,i], alpha=.8, k_size=25, g_std=10) for i in range(3)], -1)
show(im)


## 2.3 Gaussian and Lapacian Stacks

Here, we just append the current image to the smaller size stack that begins with the blurred current image

In [ ]:
def gaussian_stack(image, levels, g_std = 1, k_size = 5):
  image = np.copy(image)
  if levels == 1:
    return [image]
  
  G = cv2.getGaussianKernel(k_size, g_std) @ cv2.getGaussianKernel(k_size, g_std).T
  if len(image.shape) > 2:
    next_image = np.stack([convolve2d(image[:,:,i], G, mode = 'same') for i in range(3)], axis =-1)
  else:
    next_image = convolve2d(image, G, mode = 'same')


  return [image] + gaussian_stack(next_image, levels - 1, g_std * 2, k_size + 7)

def laplacian_stack(image, levels, g_std = 1, k_size = 5):
  stack = gaussian_stack(image, levels, g_std, k_size)

  for i in range(len(stack) - 1):
    stack[i] = stack[i] - stack[i+1]

  return stack


Gaussian stack in action!

In [ ]:
stack = gaussian_stack(cameraman[:,:,0], 5, g_std = 2, k_size=21)

fig, axs = fig, ax = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(stack[i], cmap = 'gray')

Laplacian Stack in action!

In [ ]:
stack = laplacian_stack(cameraman[:,:,0], 5, g_std = 2, k_size=21)

fig, axs = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(stack[i], cmap = 'gray')

In [ ]:
orange_stack = laplacian_stack(get_image('orange.jpeg')/255, 5, g_std = 2, k_size = 21)
apple_stack = laplacian_stack(get_image('apple.jpeg')/255, 5, g_std = 2, k_size = 21)

fig, axs = plt.subplots(3, 2, figsize=(15, 15))
axs[0, 0].set_title("Level 0")
axs[0, 0].imshow(apple_stack[0] * 10)
axs[1, 0].set_title("Level 2")
axs[1, 0].imshow(apple_stack[2] * 5)
axs[2, 0].set_title("Level 4")
axs[2, 0].imshow(apple_stack[4])
axs[0, 1].set_title("Level 0")
axs[0, 1].imshow(orange_stack[0] * 10)
axs[1, 1].set_title("Level 2")
axs[1, 1].imshow(orange_stack[2] * 5)
axs[2, 1].set_title("Level 4")
axs[2, 1].imshow(orange_stack[4])

## 2.4 Multiresolution Blending

Basically, we just take gaussian pyramid of the mask, and the stacks of the two images, and blend by the amount specified in the mask, adding them all to a matrix of zeros to get our final image

In [ ]:
def blend(positive, negative, mask, levels, g_std, k_size):
  mask_stack = gaussian_stack(mask, levels, g_std, k_size)
  negative_stack = laplacian_stack(negative, levels, g_std, k_size)
  positive_stack = laplacian_stack(positive, levels, g_std, k_size)

  image = np.zeros_like(mask)
  for i in range(len(mask_stack)):
    image += positive_stack[i] * mask_stack[i] + negative_stack[i] * (1 - mask_stack[i])

  return image

Here is the mask that we are going to take the laplacian pyramid of and elementwise multiply by

In [ ]:
mask = np.zeros_like(get_image('orange.jpeg'))
for i in range(int(len(mask[0])/2)):
  mask[:,i,:] = 1
show(mask)


Done!

In [ ]:
orange = get_image('orange.jpeg')/255
apple = get_image('apple.jpeg')/255

show(blend(apple, orange, mask, levels= 5, g_std = 2, k_size = 21))


Other Blends

In [ ]:
beautiful_world = get_image('beautiful_world.jpg')/255
temple = get_image('temple.jpg')/255
mask = np.zeros_like(beautiful_world)
mask[:380, 200:550, :] = 1

mask = np.pad(mask, ((0, 564), (0, 1020), (0, 0)))
beautiful_world = np.pad(beautiful_world, ((0, 564), (0, 1020), (0, 0)))

show(beautiful_world * mask)
show(temple)

In [ ]:
levels= 5
g_std = 4
k_size = 21

positive = beautiful_world
negative = temple

mask_stack = gaussian_stack(mask, levels, g_std * 2, k_size * 2 + 1)
negative_stack = laplacian_stack(negative, levels, g_std, k_size)
positive_stack = laplacian_stack(positive, levels, g_std, k_size)

positive_masked_stack = []
negative_masked_stack = []

image = np.zeros_like(mask)
for i in range(len(mask_stack)):
  temp1 = positive_stack[i] * mask_stack[i]
  positive_masked_stack.append(temp1)
  temp2 = negative_stack[i] * (1 - mask_stack[i])
  negative_masked_stack.append(temp2)
  image += temp1 + temp2


In [ ]:
fig, axs = fig, ax = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(mask_stack[i], cmap = 'gray')

In [ ]:
show(image, figsize=50)

In [ ]:
beautiful_world = get_image('beautiful_world.jpg')/255
miracles = get_image('miracles.jpg')/255
mask = np.zeros_like(beautiful_world)
mask[40:350, 220:510, :] = 1

mask = np.pad(mask, ((0, 49), (210, 170), (0, 0)))
beautiful_world = np.pad(beautiful_world, ((0, 49), (210, 170), (0, 0)))

show(beautiful_world * mask)
show(miracles)

In [ ]:
levels= 5
g_std = 4
k_size = 21

positive = beautiful_world
negative = miracles

mask_stack = gaussian_stack(mask, levels, g_std * 4, k_size * 4 + 1)
negative_stack = laplacian_stack(negative, levels, g_std, k_size)
positive_stack = laplacian_stack(positive, levels, g_std, k_size)

positive_masked_stack = []
negative_masked_stack = []

image = np.zeros_like(mask)
for i in range(len(mask_stack)):
  temp1 = positive_stack[i] * mask_stack[i]
  positive_masked_stack.append(temp1)
  temp2 = negative_stack[i] * (1 - mask_stack[i])
  negative_masked_stack.append(temp2)
  image += temp1 + temp2


In [ ]:
fig, axs = fig, ax = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(mask_stack[i], cmap = 'gray')

In [ ]:
show(image, figsize=50)

In [ ]:
fig, axs = fig, ax = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(positive_masked_stack[i], cmap = 'gray')

In [ ]:
fig, axs = fig, ax = plt.subplots(1, 5, figsize=(24, 50))
for i in range(5):
  axs[i].imshow(negative_masked_stack[i], cmap = 'gray')

# Misc